**Importing important python Libraries**
* pandas
* numpy
* sklearn(GradientBoostingRegressor, train_test_split, LabelEncoder)
* network
* heapq
* math(radians, sin, cos, sqrt, atan2)

In [9]:
import heapq
import joblib
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

from scipy.spatial import cKDTree

from math import radians, sin, cos, sqrt, atan2

**Loading dataset (delivery_five_cities_tanzania.csv)**

In [6]:
#load your data from a file (e.g., CSV).
file_path = './delivery_five_cities_tanzania.csv'

print("Step 1: Loading dataset...")
df = pd.read_csv(file_path)


Step 1: Loading dataset...


**Processing and Feature Enginering**

In [7]:

print("Step 2: Preprocessing and Feature Engineering...")

# Convert the string dates to a datetime objects
df['sign_time'] = pd.to_datetime(df['sign_time'], format='%m-%d %H:%M:%S')
df['receipt_time'] = pd.to_datetime(df['receipt_time'], format='%m-%d %H:%M:%S')

# Calculate the target variable: travel time in minutes
df['travel_time_minutes'] = (df['sign_time'] - df['receipt_time']).dt.total_seconds() / 60

# Extract temporal features
df['hour'] = df['receipt_time'].dt.hour
df['day_of_week'] = df['receipt_time'].dt.dayofweek

# Haversine distance function to calculate straight-line distance (in km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Calculate straight-line distance using the haversine() function
df['distance_km'] = df.apply(
    lambda row: haversine(row['receipt_lat'], row['receipt_lng'], row['sign_lat'], row['sign_lng']),
    axis=1
)

# Encode categorical features
le = LabelEncoder()
df['city_encoded'] = le.fit_transform(df['from_city_name'])
df['typecode_encoded'] = le.fit_transform(df['typecode'])

# Define features and target
features = ['receipt_lng', 'receipt_lat', 'sign_lng', 'sign_lat', 'hour', 'day_of_week', 'distance_km', 'city_encoded', 'typecode_encoded']
target = 'travel_time_minutes'

X = df[features]
y = df[target]


Step 2: Preprocessing and Feature Engineering...


**Model Selection and Training**

In [ ]:
print("Step 3: Training the Gradient Boosting Regressor model...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

joblib.dump(model, 'parcel_wise_model.joblib')

**Building the Hybrid System with a Graph Algorithm**

In [5]:
print("Step 4: Building the hybrid model...")

# Create a graph from your data
G = nx.DiGraph()

# Add nodes (locations) to the graph. We use a tuple of (lng, lat) as a unique node ID.
for index, row in df.iterrows():
    start_node = (row['receipt_lng'], row['receipt_lat'])
    end_node = (row['sign_lng'], row['sign_lat'])
    G.add_node(start_node)
    G.add_node(end_node)
    
# Add edges with weights (actual travel time for demonstration)
for index, row in df.iterrows():
    start_node = (row['receipt_lng'], row['receipt_lat'])
    end_node = (row['sign_lng'], row['sign_lat'])
    G.add_edge(start_node, end_node, weight=row['travel_time_minutes'])

Step 4: Building the hybrid model...


**Visualizing the graph**

In [6]:
def export_graph(graph, filename="exported_graph.gexf"):
    """
    Exports the graph to a GEXF file for visualization in other tools.
    
    Args:
        graph (networkx.DiGraph): The graph to export.
        filename (str): The name of the output file.
    """
    try:
        nx.write_gexf(graph, filename)
        print(f"Graph successfully exported to '{filename}'")
        print("You can now open this file in a tool like Gephi or Cytoscape.")
    except Exception as e:
        print(f"Error exporting graph: {e}")

print("Step 5: Exporting the graph to a GEXF file...")
# We will export the small sample graph for demonstration purposes.
# For your full graph, you would call `export_graph(G)` after building it.
export_graph(G, "sample_graph.gexf")
print("\n" + "="*50 + "\n")

Step 5: Exporting the graph to a GEXF file...
Graph successfully exported to 'sample_graph.gexf'
You can now open this file in a tool like Gephi or Cytoscape.




In [7]:
# Define the prediction function to be used as a dynamic edge weight
def predict_travel_time(start_loc, end_loc, timestamp, features_model):
    """
    This function simulates using the trained ML model to predict travel time
    for a new route segment.
    """
    # Create a dummy DataFrame with the new features
    new_data = pd.DataFrame([{
        'receipt_lng': start_loc[0],
        'receipt_lat': start_loc[1],
        'sign_lng': end_loc[0],
        'sign_lat': end_loc[1],
        'hour': timestamp.hour,
        'day_of_week': timestamp.dayofweek,
        'distance_km': haversine(start_loc[1], start_loc[0], end_loc[1], end_loc[0]),
        'city_encoded': 0, # Placeholder, as we don't know the city for a new route
        'typecode_encoded': 0, # Placeholder
    }])
    
    # Predict the travel time
    predicted_time = features_model.predict(new_data[features])[0]
    return max(0, predicted_time) # Ensure predicted time is not negative

# Implement the A* search algorithm
def a_star_search(graph, start_node, end_node, timestamp, model):
    """
    A* search algorithm to find the shortest path based on predicted travel times.
    """
    # Priority queue to store nodes to visit
    queue = [(0, start_node, [start_node])]
    # Set to keep track of visited nodes to avoid cycles
    visited = {start_node: 0}
    
    while queue:
        (cost, current_node, path) = heapq.heappop(queue)
        
        # Goal check
        if current_node == end_node:
            return path, cost
        
        # Explore neighbors
        for neighbor in graph.neighbors(current_node):
            
            # Predict travel time for the edge using our ML model
            edge_weight = predict_travel_time(current_node, neighbor, timestamp, model)
            new_cost = cost + edge_weight
            
            if neighbor not in visited or new_cost < visited[neighbor]:
                visited[neighbor] = new_cost
                new_path = path + [neighbor]
                # Heuristic: straight-line distance to the end node
                heuristic_cost = haversine(neighbor[1], neighbor[0], end_node[1], end_node[0])
                heapq.heappush(queue, (new_cost + heuristic_cost, neighbor, new_path))
                
    return None, float('inf')

**Implement a function which takes consideration of new location before performing A star search algorithm**

In [8]:

def find_optimal_path_for_new_delivery(graph, start_loc, end_loc, timestamp, model, num_neighbors=3):
    """
    Finds the optimal path for a new delivery that may have locations not in the graph.
    It does this by augmenting the graph with the new locations and connections
    to the nearest known points in the network.
    """
    # Create a temporary copy of the graph to avoid modifying the original
    temp_graph = graph.copy()

    # Add new start and end nodes if they don't already exist
    if start_loc not in temp_graph:
        temp_graph.add_node(start_loc)
    if end_loc not in temp_graph:
        temp_graph.add_node(end_loc)
        
    # Find the nearest neighbors to the new locations
    existing_nodes = list(graph.nodes())
    start_neighbors = sorted(existing_nodes, key=lambda loc: haversine(start_loc[1], start_loc[0], loc[1], loc[0]))[:num_neighbors]
    end_neighbors = sorted(existing_nodes, key=lambda loc: haversine(end_loc[1], end_loc[0], loc[1], loc[0]))[:num_neighbors]
    
    # Add virtual edges from the new start node to its nearest neighbors
    for neighbor in start_neighbors:
        weight = predict_travel_time(start_loc, neighbor, timestamp, model)
        temp_graph.add_edge(start_loc, neighbor, weight=weight)
        
    # Add virtual edges from the nearest neighbors to the new end node
    for neighbor in end_neighbors:
        weight = predict_travel_time(neighbor, end_loc, timestamp, model)
        temp_graph.add_edge(neighbor, end_loc, weight=weight)
        
    # Run the A* search on the augmented graph
    return a_star_search(temp_graph, start_loc, end_loc, timestamp, model)

**Demonstrating the hybrid model**

In [ ]:
print("\nStep 5: Demonstrating the hybrid model...")

#Locations are in the dataset but probably not in the training data
start_location = (-8.923860028735865,33.4214411329554)
end_location = (-3.3714501814517206,36.66093210406623)

print(f"Finding the optimal route for existing locations: {start_location} -> {end_location}...")
current_timestamp = pd.to_datetime('2025-09-01 14:00')
optimal_path, total_time = find_optimal_path_for_new_delivery(G, start_location, end_location, current_timestamp, model)
if optimal_path:
    print("\nOptimal Path (based on ML predictions):")
    for i, node in enumerate(optimal_path):
        print(f"  {i+1}. {node}")
    print(f"\nTotal Predicted Travel Time: {total_time:.2f} minutes")
else:
    print("\nNo path found between the specified locations.")

print("\n" + "="*50 + "\n")

# Example 2: New locations not in the dataset
# These coordinates are intentionally new to the graph
start_location_new = (12.9800, 33.420)
end_location_new = (12.9600, 77.5800)

print(f"Finding the optimal route for NEW locations: {start_location_new} -> {end_location_new}...")
optimal_path_new, total_time_new = find_optimal_path_for_new_delivery(G, start_location_new, end_location_new, current_timestamp, model)

if optimal_path_new:
    print("\nOptimal Path for NEW Locations:")
    for i, node in enumerate(optimal_path_new):
        print(f"  {i+1}. {node}")
    print(f"\nTotal Predicted Travel Time: {total_time_new:.2f} minutes")
else:
    print("\nNo path found between the specified new locations.")



Step 5: Demonstrating the hybrid model...
Finding the optimal route for existing locations: (-8.923860028735865, 33.4214411329554) -> (-3.3714501814517206, 36.66093210406623)...

Optimal Path (based on ML predictions):
  1. (-8.923860028735865, 33.4214411329554)
  2. (32.86782089052361, -2.466427364319329)
  3. (-3.3714501814517206, 36.66093210406623)

Total Predicted Travel Time: 6028.11 minutes


Finding the optimal route for NEW locations: (12.98, 33.42) -> (12.96, 77.58)...

Optimal Path for NEW Locations:
  1. (12.98, 33.42)
  2. (32.86786736066511, -2.466413007056262)
  3. (12.96, 77.58)

Total Predicted Travel Time: 9270.63 minutes
